In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn as nn

In [4]:
data = pd.read_csv('test.csv').set_index(['timestamp','ticker'])
data

,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,pct_change_shift
timestamp,ticker,,,,,,,,,,
2019-09-23,1,15.34,15.47,15.18,15.38,15.3122,1403282.00,0.2608,0.7428,1.5038,2.405722
2019-09-24,1,15.47,15.54,15.18,15.18,15.2886,1129103.24,-1.3004,0.5977,1.2100,3.491436
2019-09-25,1,15.20,15.85,15.19,15.75,15.5996,1938892.72,3.7549,1.0263,2.0778,0.952381
2019-09-26,1,15.86,16.04,15.66,15.71,15.8491,1686592.58,-0.2540,0.8928,1.8074,-0.763845
2019-09-27,1,15.85,16.00,15.70,15.90,15.8634,1007706.27,1.2094,0.5193,1.0235,1.886792
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-15,1,19.31,20.99,19.19,19.52,19.4779,841618.41,0.7744,0.4337,0.9785,-5.071721
2021-09-16,1,19.30,19.50,18.73,19.08,18.9852,1176953.56,-2.2541,0.6065,1.3684,-5.765199
2021-09-17,1,18.90,19.00,18.41,18.53,18.6572,1124879.63,-2.8826,0.5797,1.3078,-4.587156


In [7]:
delay = 30 # 设置delay的天数
delay_data = data.shift(delay)
delay_data.to_csv('delay.csv')
delay_data

,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,pct_change_shift
timestamp,ticker,,,,,,,,,,
2019-09-23,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-24,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-25,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-26,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-27,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-15,1,18.04,18.07,17.60,17.81,17.8197,729620.08,-0.4472,0.3760,0.8483,0.280741
2021-09-16,1,17.70,18.05,17.58,17.68,17.7759,621573.88,-0.7299,0.3203,0.7227,7.805430
2021-09-17,1,17.55,17.90,17.45,17.86,17.7260,567454.13,1.0181,0.2924,0.6597,10.470325


In [22]:
array = np.array(data)
delay_array = np.array(delay_data)
day = 30
x , y , x_delay = [] , [], []
# for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
for i in range(delay,delay + 4 ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
    x.append(array[i:i+day,:-1].T)
    y.append(array[i+day-1][-1])
    x_delay.append(delay_array[i:i+day,:-1].T)
x  , y , x_delay= np.array(x) , np.array(y).reshape(-1,1) , np.array(x_delay) # x = (153, 9, 30) , y = (153,1)
x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2]) # x = (153, 1, 9, 30)
x_delay = x_delay.reshape(x_delay.shape[0],1,x_delay.shape[1],x_delay.shape[2]) # x = (153, 1, 9, 30)
x_delay = (x-x_delay)/x_delay

C:\Users\Xinfei\AppData\Local\Temp/ipykernel_9524/3492420914.py:13: RuntimeWarning: divide by zero encountered in true_divide
  x_delay = (x-x_delay)/x_delay


In [153]:
stride = 10
# 为了cov 和 corr提取并行计算产生计算列表
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0],i] for i in l1[1:]]
    l1 = l1[1:]
    return v+generateC(l1)
def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num , num_rev
def data_info(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length = data.shape[3] # 30
    feat_num = data.shape[2] # 9
    """num:组合数对列表,num_rev:num的翻转列表"""
    num , num_rev = generate_Num_and_ReversedNum(feat_num)
    conv_feat = len(num)
    # 构建步长列表，如果数据长度不能整除，则取剩下长度，如果剩下长度小于5，则与上一步结合一起
    if data_length % stride == 0:
        step_list = list(range(0,data_length+stride,stride))
    elif data_length % stride<=5:
        mod = data_length % stride
        step_list = list(range(0,data_length-stride,stride))+[data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0,data_length+stride-mod,stride))+[data_length]
    return data_length,feat_num,conv_feat,num,num_rev,step_list
def ts_cov4d(data,stride):
    '''计算4维数据的协方差'''
    '''data:[N,C,H,W],,W:price length,N:batch size'''
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    #计算的过程中务必保持keepdims=True
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,num,start:end] # (2000, 1, 36, 2, 10)
        sub_data2 = data[:,:,num_rev,start:end]
        mean1 = sub_data1.mean(axis = 4,keepdims = True) # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis = 4,keepdims = True)
        spread1 = sub_data1 - mean1 # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1*spread2).sum(axis = 4,keepdims = True)/(sub_data1.shape[4] - 1)).mean(axis = 3,keepdims = True) # (2000, 1, 36, 1, 1)
        l.append(cov)
    corr = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,conv_feat,len(step_list)-1) # (2000, 1, 36, 3)
    return torch.from_numpy(corr)
def ts_corr4d(data,stride):
    '''计算4维数据的相关系数'''
    '''data:[N,C,H,W],,W:price length,N:batch size'''
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,num,start:end]
        sub_data2 = data[:,:,num_rev,start:end]
        std1 = sub_data1.std(axis = 4,keepdims = True)
        std2 = sub_data2.std(axis = 4,keepdims = True)
        std = (std1*std2).mean(axis = 3,keepdims = True)
        l.append(std)
    std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,conv_feat,len(step_list)-1)
    cov = ts_cov4d(data,stride)
    fct = (sub_data1.shape[4]-1)/sub_data1.shape[4]
    return (cov/torch.from_numpy(std))*fct
def ts_stddev4d(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,:,start:end]
        std1 = sub_data1.std(axis = 3,keepdims = True)
        l.append(std1)
    std = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,feat_num,len(step_list)-1)
    return torch.from_numpy(std)
def ts_zscore(data,stride):
    if len(data.shape)!=4:
        raise Exception('Input data dimensions should be [N,C,H,W]')
    data_length , feat_num , conv_feat, num , num_rev,step_list= data_info(data,stride)
    l = []
    for i in range(len(step_list)-1):
        start = step_list[i]
        end = step_list[i+1]
        sub_data1 = data[:,:,:,start:end]
        mean = sub_data1.mean(axis = 3,keepdims = True)
        std = sub_data1.std(axis = 3,keepdims = True)
        z_score = mean/std
        l.append(z_score)
    z_score = np.squeeze(np.array(l)).transpose(1,2,0).reshape(-1,1,feat_num,len(step_list)-1)
    return torch.from_numpy(z_score)

In [154]:
batch = nn.BatchNorm2d(1,affine=True)
conv1 = ts_cov4d(x,10).to(torch.float)
bc1 = batch(conv1)
conv2 = ts_corr4d(x,10).to(torch.float)
bc2 = batch(conv2)
conv3 = ts_stddev4d(x,10).to(torch.float)
bc3 = batch(conv3)
conv4 = ts_zscore(x,10).to(torch.float)
bc4 = batch(conv4)
bc4

tensor([[[[ 0.5429,  0.9426, -0.1324],
          [ 0.5141,  1.0992, -0.2233],
          [ 0.3457,  1.6031, -0.1741],
          [ 0.2984,  1.2717, -0.1918],
          [ 0.4079,  1.4469, -0.2066],
          [-0.8575, -0.8129, -0.8277],
          [-0.9127, -0.9018, -0.8927],
          [-0.8575, -0.8129, -0.8277],
          [-0.8574, -0.8129, -0.8277]]],


        [[[ 0.1321,  2.3847,  0.1535],
          [ 0.1037,  2.0283,  0.1319],
          [ 0.0299,  2.5431,  0.2341],
          [ 0.0352,  1.8600,  0.3140],
          [ 0.0384,  2.6640,  0.2267],
          [-0.8573, -0.8083, -0.8397],
          [-0.9107, -0.8996, -0.8962],
          [-0.8573, -0.8083, -0.8397],
          [-0.8573, -0.8083, -0.8397]]]], grad_fn=<NativeBatchNormBackward>)

In [126]:
test = x[0][0]
num = [0,10,20,30]
for time in range(len(num) - 1):
    for i in range(test.shape[0]):
            print(np. (test[i,num[time]:num[time+1]]))
#             print(pd.DataFrame(test[[i,j],num[time]:num[time+1]].T).corr().iloc[0,1])
            print("________________________________________________")

0.2340598214132446
________________________________________________
0.2401020616321322
________________________________________________
0.2673200329193451
________________________________________________
0.2802213410859351
________________________________________________
0.25701352279598066
________________________________________________
352014.12266699696
________________________________________________
1.2616693901731941
________________________________________________
0.18140541998518125
________________________________________________
0.4092543805507767
________________________________________________
0.17472549899771372
________________________________________________
0.1620617166390632
________________________________________________
0.12737739202856974
________________________________________________
0.1481080686525891
________________________________________________
0.13685220641261184
________________________________________________
137800.56595758555
________________________